In [37]:
import pandas as pd
import numpy as np
import Expedia_Transformer.Part_2_dataframe_preprocess as pre
import Expedia_Transformer.true_model as mt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Use all data to train Transformer

In [38]:
df = pd.read_csv("./Expedia_preprocess/df_OR_normal_process_with_nonpurchase_clean.csv")

In [39]:
weight_index = np.sum(df['booking_bool'])/(len(list(df['srch_id'].unique()))-np.sum(df['booking_bool']))
if weight_index > 1:
    pt_weight = [1,int(weight_index)]
else:
    pt_weight = [int(1/weight_index),1]

In [40]:
mt.train_model(df,pt_weight = pt_weight,model_name = "true_model")

---------------- Epoch: 01 ----------------
Epoch 0 loss 1.4487127588496167
valid loss 1.36457
---------------- Epoch: 02 ----------------
Epoch 1 loss 1.3581021075225428
valid loss 1.36507
---------------- Epoch: 03 ----------------
Epoch 2 loss 1.355908168220364
valid loss 1.36419
---------------- Epoch: 04 ----------------
Epoch 3 loss 1.3569547268755935
valid loss 1.36431
---------------- Epoch: 05 ----------------
Epoch 4 loss 1.356329837729949
valid loss 1.37025
---------------- Epoch: 06 ----------------
Epoch 5 loss 1.357066922043618
valid loss 1.36411
---------------- Epoch: 07 ----------------
Epoch 6 loss 1.3557421126280724
valid loss 1.3656
EarlyStopping counter: 1 out of 3
---------------- Epoch: 08 ----------------
Epoch 7 loss 1.3559739310760757
valid loss 1.36477
EarlyStopping counter: 2 out of 3
---------------- Epoch: 09 ----------------
Epoch 8 loss 1.3572163769217997
valid loss 1.36427
EarlyStopping counter: 3 out of 3
Early stopping
---------------- Epoch: 01 -----

In [19]:
df = pd.read_csv("./Expedia_preprocess/df_OR_normal_process_with_nonpurchase_clean.csv")

In [21]:
sum(df['booking_bool'])

41485

# Generate Data for True Model to calculate RMSE and top accuracy

In [22]:
unique_srch_ids = df['srch_id'].unique()
order_list_random = np.random.permutation(unique_srch_ids)
train_list = order_list_random[:int(len(order_list_random)*0.9)]
test_list = order_list_random[int(len(order_list_random)*0.9):]

In [23]:
df_train = df[df['srch_id'].isin(train_list)]
df_test = df[df['srch_id'].isin(test_list)]

In [24]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.to_csv("./True_Model/df_train.csv",index=False)
df_test.to_csv("./True_Model/df_test.csv",index=False)

In [4]:
df_train = pd.read_csv("./True_Model/df_train.csv")


# Train Test Model

In [5]:
mt.train_model(df_train,pt_weight = pt_weight,model_name = "test_model")

---------------- Epoch: 01 ----------------
Epoch 0 loss 1.3138690426852893
valid loss 1.23499
---------------- Epoch: 02 ----------------
Epoch 1 loss 1.220888506006641
valid loss 1.23341
---------------- Epoch: 03 ----------------
Epoch 2 loss 1.2201944069978103
valid loss 1.23381
---------------- Epoch: 04 ----------------
Epoch 3 loss 1.219359732075351
valid loss 1.23222
---------------- Epoch: 05 ----------------
Epoch 4 loss 1.2194578747221863
valid loss 1.23344
---------------- Epoch: 06 ----------------
Epoch 5 loss 1.2193737759177021
valid loss 1.232
---------------- Epoch: 07 ----------------
Epoch 6 loss 1.2205373115366158
valid loss 1.23776
EarlyStopping counter: 1 out of 3
---------------- Epoch: 08 ----------------
Epoch 7 loss 1.2192777245543736
valid loss 1.23456
EarlyStopping counter: 2 out of 3
---------------- Epoch: 09 ----------------
Epoch 8 loss 1.2198740513936623
valid loss 1.23459
EarlyStopping counter: 3 out of 3
Early stopping
---------------- Epoch: 01 -----

In [24]:
df_test = pd.read_csv("./True_Model/df_test.csv")

In [25]:
sum(df_test['booking_bool'])

4107

In [26]:
# final_result = mt.predict(df_test, max_num=38,model_name = "test_model", true_model_flag=True)
final_result = mt.predict(df_test, max_num=38,model_name = "test_model")

In [27]:
final_result[0, 0:18].squeeze().detach().numpy()

array([0.2715014 , 0.28786978, 0.02445496, 0.11775255, 0.12682259,
       0.0029957 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ], dtype=float32)

In [32]:
df_test = pd.read_csv("./True_Model/df_test.csv")

In [34]:
count = 0
number_count = 0
buy_count =0
top_1_acc = 0
top_3_acc = 0
total_RMSE = []
for group_id, group_df in df_test.groupby('srch_id'):
    temp_list = np.array(group_df['booking_bool'])
    predicted = final_result[count, 0:(len(temp_list))].squeeze().detach().numpy()
#     print(temp_list.argmax())
#     print(predicted.argmax())
#     print(sum(temp_list))
    total_RMSE.append(np.sum((temp_list - predicted)**2) + (1-np.sum(temp_list)-(1-np.sum(predicted)) )**2)
    if sum(temp_list)==1:
        top_1_acc += (temp_list.argmax()== predicted.argmax())
        for turn in range(3):
            top_3_acc += (temp_list.argmax()== predicted.argmax())
            predicted[predicted.argmax()] = 0
        buy_count += 1
    number_count += len(temp_list)
    count += 1

In [35]:
np.sqrt(sum(total_RMSE)/(number_count+count))

0.18741832354474694

In [36]:
top_1_acc/buy_count, top_3_acc/buy_count

(0.18431945458972487, 0.40589237886535184)